## IMPORTS

In [19]:
import cv2
from skimage.metrics import structural_similarity as compare_ssim
import os
import time
import numpy as np

from scipy.spatial import distance as dist
import dlib
from imutils import face_utils
import imutils

import itertools
import matplotlib.pyplot as plt



## Selfie Utils

In [20]:
def edit_img(frame):
    '''
    This functions gets an image and does the following:
        1. resizes the image
        2. changes the image to gray-scale

    Input:
        frame   numpy array
    '''
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return gray, frame

In [21]:
def take_regular_selfie(frame, total, output_folder):
    '''

    '''
    img_name = os.path.join(output_folder, "selfie_frame_{}.png".format(total))

    cv2.imwrite(img_name, frame)
    print("{} written!".format(img_name))

    return 

In [22]:
def mouth_aspect_ratio(mouth):
    '''

    '''
    A = dist.euclidean(mouth[3], mouth[9])
    B = dist.euclidean(mouth[2], mouth[10])
    C = dist.euclidean(mouth[4], mouth[8])
    avg = (A + B + C) / 3
    D = dist.euclidean(mouth[0], mouth[6])
    mar = avg/D
    return mar #return the mouth aspect ratio

In [23]:
def eye_aspect_ratio(eye):
    '''

    '''
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])

    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)

    # return the eye aspect ratio
    return ear

In [24]:
def init_model():
    print("Initializing model")
    shape_predictor= "dat_files/shape_predictor_68_face_landmarks.dat" #dace_landmark
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(shape_predictor)
    return detector, predictor

In [25]:
def detect_face(np_img, detector, predictor):
    '''

    '''
    faces = []
    mouth_start, mouth_end = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]
    l_eye_start, l_eye_end = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    r_eye_start, r_eye_end = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

    rects = detector(np_img, 0)
    for rect in rects:
        shape = predictor(np_img, rect)
        shape = face_utils.shape_to_np(shape)
        mouth = shape[mouth_start:mouth_end]
        l_eye = shape[l_eye_start:l_eye_end]
        r_eye = shape[r_eye_start:r_eye_end]
        mar = mouth_aspect_ratio(mouth)
        l_ear = eye_aspect_ratio(l_eye)
        r_ear = eye_aspect_ratio(r_eye)

        faces.append({'mar': mar,
                      'mouth_hull' : cv2.convexHull(mouth),
                      'l_ear' : l_ear, 
                      'l_eye_hull' : cv2.convexHull(l_eye),
                      'r_ear' : r_ear,
                      'r_eye_hull' : cv2.convexHull(r_eye)})
    return faces

## Similiarity

In [26]:
def show_imgs(imgs_list, imgs_name_list=None, params=None, save=False):
    '''
    This func gets images and plots them side by side
    Input:
        imgs_list       np.array list       list of imgs to plot
        imgs_name_list  list of str         list of imgs name
        params          str                 str to describe running params to be used as title    
    '''
    imgs_num = len(imgs_list)
    if imgs_num == 4:
        fig, axarr = plt.subplots(2, 2)
        i_range = 2
        j_range = 2
    
        counter = 0
        for i in range(i_range):
            for j in range(j_range):
                axarr[i][j].imshow(imgs_list[counter], cmap='gray')
                if imgs_name_list is not None:
                    axarr[i][j].set_title(imgs_name_list[counter])
                counter += 1

    else:
        fig, axarr = plt.subplots(1, imgs_num)
        i_range = imgs_num

        for i in range(i_range):
            axarr[i].imshow(imgs_list[i], cmap='gray')
            if imgs_name_list is not None:
                axarr[i].set_title(imgs_name_list[i])

    if params is not None:
        fig.suptitle(params)

    save =False
    if save:
        params = params.split('\n')[0]
        plt.savefig("plots/" + params + ".png")

    else:
        plt.show()
    plt.close()


In [27]:
def mse(img1, img2):
    '''
    This func cumputes Mean Squared Error between the two images

    Input:
        img1        np.array
        img2        np.array
          
    Output:
        err         float       diff between images          
    '''

    err = np.sum((img1 - img2) ** 2)
    err /= float(img1.shape[0] * img2.shape[1])
	
    return err

In [28]:
def compare_sim(img1, img2, method=None, debug=False):
    '''
    This func checks whether two images are similiar

    Input:
        img1        np.array
        img2        np.array
        method      str         method for similarity test. it can be MSE or ssim
          
    Output:
        similarity  bool        whether two images are similar
    '''
    
    ssim_sim = True
    mse_sim = True

    (score, diff) = compare_ssim(img1, img2, full=True)
    if score < 0.75:
        ssim_sim = False
    err = mse(img1, img2)
    if err > 50.0:
        mse_sim = False

    if debug:
        img3 = img1 - img2 #diff in MSE
        img4 = diff
        show_imgs([img1, img2, img3, img4], params=f"ssim score is: {score:.2f} and MSE err is: {err:.2f}")

    return np.logical_and(ssim_sim, mse_sim)

## Cartoonifier

In [29]:
def edge_mask(img, line_size, blur_value):
  '''
  This functions creates an edge mask for given image using canny and dilation

  Input:
      img       numpy array     given image to create its edge mask

  Output:
      edges     numpy array     the edge mask of the image, with dilation
  '''


  ##################### start part of try using canny #####################
  '''
  # sigma, L_th, H_th = 1, 0.05, 0.27
  
  # edges = canny.cannyEdges(gray, sigma, L_th, H_th)
  
  # edges = edges.astype(np.uint8)
  # edges = np.logical_not(edges)
  # edges = edges*255
  
  # edges = edges.astype(np.uint8)
  '''
  ##################### end part of try usin canny #####################

  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  gray_blur = cv2.medianBlur(gray, blur_value)
  edges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur_value)

  return edges

In [30]:
def color_quantization(img, k=9):

  '''
  This functions reduce the number of colors to a given image

  Input:
      img       numpy array     image to reduce its colors
      k         int             number of new different colors


  Output:
      result     numpy array     image recolored to k different colors
  '''

# Transform the image
  data = np.float32(img).reshape((-1, 3))

# Determine criteria
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.001)

# Implementing K-Means
  ret, label, center = cv2.kmeans(data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
  center = np.uint8(center)
  result = center[label.flatten()]
  result = result.reshape(img.shape)
  return result

In [31]:
def cartoonify(img, total, output_folder):
  '''
  This functions make a cartoon out of a given image and saves the cartoon in a given path

  Input:
      img               numpy array       image to create is cartoon
      total             int               counter of number of selfies - used as id
      output_folder     string            path to save the selfies in

  '''

  line_size = 7
  blur_value = 7

  edges = edge_mask(img, line_size, blur_value)
  img = color_quantization(img)

  ##################### start part of try using canny #####################
  '''
  # edges = np.repeat(edges[:, :, np.newaxis], 3, axis=2)
  # cartoon = np.bitwise_and(img, edges)
  '''
  ##################### end part of try using canny #####################

  blurred = cv2.bilateralFilter(img, d=7, sigmaColor=200, sigmaSpace=200)
  cartoon = cv2.bitwise_and(blurred, blurred, mask=edges)

  img_name = os.path.join(output_folder, "selfie_cartoon_{}.png".format(total))

  cv2.imwrite(img_name, cartoon)

  print("{} written!".format(img_name))

## Pencilifier

In [32]:
def pencilMe(img, total, output_folder):
    '''
    This functions make a pancil sketch out of a given image and saves the sketch in a given path.
    this is done by:
        - Converting an image into gray_scale image
        - Inverting the image
        - Smoothing the image
        - Obtaining the final sketch

    Input:
        img               numpy array       image to create is cartoon
        total             int               counter of number of selfies - used as id
        output_folder     string            path to save the selfies in

    '''
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_invert = cv2.bitwise_not(img_gray)
    img_smoothing = cv2.GaussianBlur(img_invert, (21, 21), sigmaX=0, sigmaY=0)
    final_img = cv2.divide(img_gray, 255 - img_smoothing, scale=256)

    img_name = os.path.join(output_folder, "selfie_pancil_{}.png".format(total))

    cv2.imwrite(img_name, final_img)
    print("{} written!".format(img_name))## pencilifier

## ALL UNITE - MAIN

In [33]:
def takeSelfies(frame, gray, output_folder, cartoon, pencil, useCascade, useDlib, useBoth, eyes_cascade = None, smile_cascade= None, faces = None):
    '''
    This functions takes the selfie if conditions of found smile and two eyes is met. the function call the desired filter function if needed

    Input:

        frame               numpy array             the current observed frame
        gray                numpy array             the current observed frame as a gray scale image
        output_folder       string                  path to save the selfies in
        cartoon             boolean                 if true - use this filter when taking selfie
        pencil              boolean                 if true - use this filter when taking selfie
        useCascade          boolean                 if true - look for features only using cascades
        useDlib             boolean                 if true - look for features only using Dlib
        useBoth             boolean                 if true - look for features using cascades and Dlib
        eyes_cascade        numpy array             array of arrays of coordinates of found eyes on image by using haar cascade.
        smile_cascade       numpy array             array of coordinates of found smile on image by using haar cascade.
        faces               dict                    dict contains features found using Dlib

    '''
    global total, counter, last_taken_selfie

    featuresDetected = False
    detectCascade = ((eyes_cascade is not None) and (smile_cascade is not None ) and (len(eyes_cascade)%2 ==0) and (len(smile_cascade)>=1))
    detectDlib = False
    if faces:
        for face in faces:
            currentFaceDetectDlib = (face['mar'] <= .26 or face['mar'] > .32) and (face['l_ear'] > .25) and (face['r_ear'] > .25)
            detectDlib = currentFaceDetectDlib or detectDlib

    if useBoth and detectCascade and detectDlib:
        featuresDetected = True

    elif useCascade and detectCascade:
        featuresDetected = True

    elif useDlib and detectDlib:
        featuresDetected = True

    if featuresDetected:
        counter += 1

        # print(f"counter is: {counter}")
        if counter >= 5:  # we need to check it
            if last_taken_selfie is not None:
                score = compare_sim(last_taken_selfie, gray)
                # print("SSIM: {}".format(score))
                # check if there is a difference between current img to last taken selfie.
                # if it is the same go back to while loop

                if score:
                    total += 1
                    if cartoon:
                        cartoonify(frame, total, output_folder)
                    elif pencil:
                        pencilMe(frame, total, output_folder)
                    else:
                        take_regular_selfie(frame, total, output_folder)
                    last_taken_selfie = gray
                    counter = 0
                pass

            else:
                # first selfie

                if cartoon:
                    cartoonify(frame, total, output_folder)
                elif pencil:
                    pencilMe(frame, total, output_folder)
                else:
                    take_regular_selfie(frame, total, output_folder)

                total += 1
                last_taken_selfie = gray
                counter = 0

                pass

In [34]:
def haarDetection(face_cascade, eye_cascade, smile_cascade, gray, img):
    '''
    This functions detect the face, eyes and smile in a given gray image and draw the detection on the colored image - all using haar cascades

    Input:
        face_cascade        Cascade Classifier      classifier that is used to detect faces in a given image
        eye_cascade         Cascade Classifier      classifier that is used to detect eyes in a given image
        smile_cascade       Cascade Classifier      classifier that is used to detect smile in a given image
        gray                numpy array             the current observed frame as a gray scale image
        img                 numpy array             the current observed frame


    Output:
        img                 numpy array             the current observed frame with the rectangles of the detect face, eyes and smile
        eyes                numpy array             array of arrays of coordinates of found eyes on image.
        smile               numpy array             array of coordinates of found smile on image.

    '''
    eyes = None
    smile = None
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x_face, y_face, w_face, h_face) in faces:
        cv2.rectangle(img, (x_face, y_face), (x_face+w_face, y_face+h_face), (255, 0, 0), 2)
        roi_gray = gray[y_face:y_face + h_face, x_face:x_face + w_face]
        roi_color = img[y_face:y_face + h_face, x_face:x_face + w_face]

        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.03, minNeighbors=40, minSize=(30,30))
        for (x_eye, y_eye, w_eye, h_eye) in eyes:
            cv2.rectangle(roi_color, (x_eye, y_eye), (x_eye + w_eye, y_eye + h_eye), (0, 255, 0), 2)


        # add from eyes pattern
        smile = smile_cascade.detectMultiScale(roi_gray, scaleFactor=1.2, minNeighbors=40, minSize=(30,30))
        for (x_smile, y_smile, w_smile, h_smile) in smile:
            cv2.rectangle(roi_color, (x_smile, y_smile), (x_smile + w_smile, y_smile + h_smile), (0, 255, 0), 2)

    return img, eyes, smile

In [35]:
def dlibDetection(frame, detector, predictor, show_stats, draw_contours):
    '''
    This functions detect the face, eyes and smile in a given image and draw the detection of the features - using Dlib

    Input:
        frame               numpy array      classifier that is used to detect faces in a given image
        detector
        predictor
        show_stats          boolean             indicates if show stats of features on image
        draw_contours       boolean             indicates if show contours of features on image


    Output:
        resized_frame       numpy array             the frame resized
        faces               dict                    dict contains features found using Dlib

    '''
    gray_img, resized_frame = edit_img(frame)
    faces = detect_face(gray_img, detector, predictor)

    i = 0  # for putText
    for face in faces:
        if draw_contours:
            for face_part in ['mouth_hull', 'l_eye_hull', 'r_eye_hull']:
                cv2.drawContours(resized_frame, [face[face_part]], -1, (0, 255, 0), 1)

        if show_stats:
            y0, dy = 30, 30
            for ar in ['mar', 'l_ear', 'r_ear']:
                y = y0 + i * dy
                cv2.putText(resized_frame, f"{ar}: {face[ar]:.5f}", (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            (0, 0, 255), 2)
                i += 1
            cv2.putText(resized_frame, f"{'mar'}: {face['mar']}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (0, 0, 255), 2)
    return resized_frame, faces

In [36]:
def run(useCascade, useDlib, useBoth, face_cascade=None, eye_cascade=None, smile_cascade=None, cartoon=False, pencil=False):
    '''
    This functions is the main function of the selfies.
    it reads the frames from the camera and take the selfie with or without filter.
    the function also lets the user can change the filter selection by clicking on the relevant letters.

    Input:
        useCascade          boolean                 if true - look for features only using cascades
        useDlib             boolean                 if true - look for features only using Dlib
        useBoth             boolean                 if true - look for features using cascades and Dlib
        face_cascade        numpy array             array of coordinates of found face on image by using haar cascade.
        eyes_cascade        numpy array             array of arrays of coordinates of found eyes on image by using haar cascade.
        smile_cascade       numpy array             array of coordinates of found smile on image by using haar cascade.
        cartoon             boolean                 if true - use this filter when taking selfie
        pencil              boolean                 if true - use this filter when taking selfie

    '''



    print("for regular selfies press 'n'\n"
          "for cartoon selfies press 'c'\n"
          "for pencil selfies press 'p'\n")

    output_folder = 'selfies_' + time.strftime("%Y_%m_%d_%H_%M_%S")
    os.mkdir(output_folder)


    print("starting looking for perfect selfie mode")

    cap = cv2.VideoCapture(0)

    final, eye, smile = None, None, None
    faces = None

    detector, predictor = init_model()


    while True:
        ret, img = cap.read()
        frame = img.copy()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        if useBoth or useDlib:
            show_stats = True
            draw_contours = True
            if useBoth:
                show_stats = False
                draw_contours = False

            final, faces = dlibDetection(frame, detector, predictor, show_stats, draw_contours)

        if useBoth or useCascade:
            final, eye, smile = haarDetection(face_cascade, eye_cascade, smile_cascade, gray, img)



        takeSelfies(frame, gray, output_folder, cartoon, pencil, useCascade, useDlib, useBoth, eye, smile, faces)

        cv2.imshow('Video', final)

        key2 = cv2.waitKey(1) & 0xFF
        if key2 == ord('q'):
            break
        if key2 == ord('p'):
            pencil = True
            cartoon = False
            print("taking *PENCIL* selfies..")
        if key2 == ord('c'):
            pencil = False
            cartoon = True
            print("taking *CARTOON* selfies..")

        if key2 == ord('n'):
            print("taking *REGULAR* selfies ")
            pencil = False
            cartoon = False

    cap.release()
    cv2.destroyAllWindows()

## Variables and Globals

In [37]:
last_taken_selfie = None
total = 0
counter = 0

useCascade = True
useDlib = True
useBoth = False

if useDlib and useCascade:
    useBoth = True
    useCascade = False
    useDlib = False

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')## Variables and Globals

## Run Main

In [38]:
run(useCascade, useDlib, useBoth, face_cascade, eye_cascade, smile_cascade, cartoon=False, pencil=False)

for regular selfies press 'n'
for cartoon selfies press 'c'
for pencil selfies press 'p'

starting looking for perfect selfie mode
Initializing model
selfies_2021_05_31_01_10_12\selfie_frame_0.png written!


TypeError: compare_sim() got an unexpected keyword argument 'full'